# Multi-model Evaluation LinkedIn Summary and FAQ

In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from pypdf import PdfReader
from pathlib import Path
from IPython.display import Markdown, display
from anthropic import Anthropic
from openai import OpenAI  # Used here to call Ollama-compatible API and Google Gemini

load_dotenv(override=True)

True

In [2]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key not set
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key exists and begins gsk_


In [6]:
anthropic = Anthropic()

# === Load PDF and extract resume text ===

reader = PdfReader("../claude_based_chatbot_tc/me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

# === Create the shared FAQ generation prompt ===
faq_prompt = (
    "Please read the following professional background and resume content carefully. "
    "Based on this information, generate a well-structured FAQ (Frequently Asked Questions) document that reflects the subject’s professional background.\n\n"
    "== RESUME TEXT START ==\n"
    f"{linkedin}\n"
    "== RESUME TEXT END ==\n\n"

    "**Instructions:**\n"
    "- Write at least 15 FAQs.\n"
    "- Each entry should be in the format:\n"
    "  - Q: [Question here]\n"
    "  - A: [Answer here]\n"
    "- Focus on real-world questions that recruiters, collaborators, or website visitors would ask.\n"
    "- Be concise, accurate, and use only the information in the resume. Do not speculate or invent details.\n"
    "- Use a professional tone suitable for publishing on a personal website.\n\n"

    "Output only the FAQ content. Do not include commentary, headers, or formatting outside of the Q/A list."
)

messages = [{"role": "user", "content": faq_prompt}]
evaluators = []
answers = []



In [ ]:
# Anthropic API Call

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
faq_prompt = claude.messages.create(
    model=model_name, 
    messages=messages, 
    max_tokens=1000
)

faq_answer = faq_prompt.content[0].text

display(Markdown(faq_answer))
evaluators.append(model_name)
answers.append(faq_answer)

In [ ]:
# === 2. Google Gemini Call ===

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

faq_prompt = gemini.chat.completions.create(model=model_name, messages=messages)
faq_answer = faq_prompt.choices[0].message.content

display(Markdown(faq_answer))
evaluators.append(model_name)
answers.append(faq_answer)


In [ ]:
# === 2. Ollama Groq Call ===

groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

faq_prompt = groq.chat.completions.create(model=model_name, messages=messages)
faq_answer = faq_prompt.choices[0].message.content

display(Markdown(faq_answer))
evaluators.append(model_name)
answers.append(faq_answer)

In [ ]:
# It's nice to know how to use "zip"

for evaluator, answer in zip(evaluators, answers):
    print(f"Evaluator: {evaluator}\n\n{answer}")


# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from evaluator {index+1}\n\n"
    together += answer + "\n\n"

In [15]:
formatter = f"""You are a meticulous AI evaluator tasked with synthesizing multiple assistant-generated career FAQs and summaries into one high-quality file. You have received {len(evaluators)} drafts based on the same resume, each containing a 2-line summary and a set of FAQ questions with answers.

---
**Original Request:**
"{faq_prompt}"
---

Your goal is to combine the strongest parts of each submission into a single, polished output. This will be the final `faq.txt` that lives in a public-facing portfolio folder.

**Evaluation & Synthesis Instructions:**

1. **Prioritize Accuracy:** Only include information clearly supported by the resume. Do not invent or speculate.
2. **Best Questions Only:** Select the most relevant and insightful FAQ questions. Discard weak, redundant, or generic ones.
3. **Edit for Quality:** Improve the clarity and fluency of answers. Fix grammar, wording, or formatting inconsistencies.
4. **Merge Strengths:** If two assistants answer the same question differently, combine the best phrasing and facts from each.
5. **Consistency in Voice:** Ensure a single professional tone throughout the summary and FAQ.

**Required Output Structure:**

1. **2-Line Summary:** Start with the best or synthesized version of the summary, capturing key career strengths.
2. **FAQ Entries:** Follow with at least 8–12 strong FAQ entries in this format:

Q: [Question]  
A: [Answer]

---
**Examples of Strong FAQ Topics:**
- Key technical skills or languages
- Past projects or employers
- Teamwork or communication style
- Remote work or leadership experience
- Career goals or current availability

This will be saved as a plain text file (`faq.txt`). Ensure the tone is accurate, clean, and helpful. Do not add unnecessary commentary or meta-analysis. The final version should look like it was written by a professional assistant who knows the subject well.
"""

formatter_messages = [{"role": "user", "content": formatter}]

In [ ]:
# === 1. Final (Claude) API Call ===
anthropic = Anthropic(api_key=anthropic_api_key)
faq_prompt = anthropic.messages.create(
    model="claude-3-7-sonnet-latest",
    messages=formatter_messages,
    max_tokens=1000,
)
results = faq_prompt.content[0].text
display(Markdown(results))


In [ ]:
gr.ChatInterface(results, type="messages").launch()